In [52]:
import numpy as np
import pandas as pd

from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor

from helper_functions import stock_prices, stock_list

In [20]:
testsmpl=126
interval = 252*3

interval = 521

In [145]:
data_historical = stock_prices.get_prices(['AAPL', 'PG'])

[*********************100%***********************]  2 of 2 completed


In [146]:
data_historical1 = data_historical.dropna(how='all')
data = data_historical1[-interval:]

In [147]:
limitPer = len(data) * .85
data = data.dropna(thresh=limitPer, axis=1)

data = np.log(data)

data = data.dropna(how='any')

data_train = data[:-testsmpl]
data_test = data[-testsmpl:]

df_train = data_train.copy()

In [172]:
def lagmat(df, T=10) -> (np.array, np.array):
    X = []
    Y = []
    tag = df.columns[0]
    # df[f'{tag}_diff'] = df[tag].diff()
    series = df[tag].to_numpy()[:]
    for t in range(len(series) - T):
        x = series[t:t+T]
        X.append(x)
        y = series[t+T]
        Y.append(y)

    X = np.array(X).reshape(-1 ,T)
    Y = np.array(Y)

    return X, Y

In [183]:
X1, Y1 = lagmat(data[['AAPL']])
X2, Y2 = lagmat(data[['PG']])

In [184]:
Y1 = np.expand_dims(Y1, axis=1)
Y2 = np.expand_dims(Y2, axis=1)

X = np.hstack((X1, X2))
Y = np.hstack((Y1, Y2))

In [187]:
X_train, X_test = X[:-testsmpl], X[-testsmpl:] 
Y_train, Y_test = Y[:-testsmpl], Y[-testsmpl:] 

In [189]:
regr = MultiOutputRegressor(MLPRegressor(random_state=123)).fit(X_train, Y_train)

In [190]:
Y_forecasts = regr.predict(X_test)

In [193]:
df_data = pd.DataFrame(Y, columns=['AAPL', 'PG'])
df_preds = pd.DataFrame(Y_forecasts, columns=['AAPL_MLP', 'PG_MLP'])

In [199]:
df_data['AAPL_MLP'] = np.nan
df_data['PG_MLP'] = np.nan

In [210]:
df_data.iloc[-Y_forecasts.shape[0]:, 2] = df_preds['AAPL_MLP']
df_data.iloc[-Y_forecasts.shape[0]:, 3] = df_preds['PG_MLP']

In [156]:
from sklearn.metrics import mean_absolute_percentage_error
def get_mape(df, ticker, pred_tag, test_count=126):
    df = df.dropna(how='any')
    test_true = df.iloc[-test_count:][ticker]
    test_pred = df.iloc[-test_count:][f'{ticker}_{pred_tag}']
    mapel = mean_absolute_percentage_error(
        test_true, test_pred
    ) 
    return mapel

In [216]:
get_mape(df_data, ticker='AAPL', pred_tag='MLP')*100

0.8898577225515905